#### This notebook helps to build the growth model from [Weiße et al. 2015](https://www.pnas.org/doi/10.1073/pnas.1416533112).

Activate the environment and load in the packages:

In [ ]:
using Pkg
Pkg.activate(joinpath(@__DIR__, ".."))

using Catalyst, DifferentialEquations, Plots, DataFrames

Like in the Michaelis-Menten example, we can start by implementing the model as a reaction network in Catalyst. Here are the reactions in the model (found the SI of the PNAS paper):


![alt text](https://raw.githubusercontent.com/hhindley/CompSysBio_Weisse/3ed46c0ddbd1fdae749b3b821abdcc9b90f7d77e/imgs/growth_model.png "Reactions")

The rates are more complicated here than the basic rate constants we were using before. You can find all the relevant rates in the SI, but here they are for convenience:

Transcription rate:

$\omega_x(a) = \omega_x \frac{a}{\theta_x + a}, \quad x \in {r,t,m}$

$\omega_q(q,a) = \omega_q \frac{a}{\theta_x + a} \frac{1}{1+(q/K_q)^{h_q}}$

Translation rate:

$\nu_x(c_x,a) = c_x \frac{\gamma(a)}{n_x}, \quad \mathrm{where} \quad \gamma(a) = \frac{\gamma_{max} a}{K_{\gamma} + a}$

Growth rate:

$\lambda = \frac{\gamma(a) (c_r + c_t + c_m + c_q)}{M}$

Energy consumption in translation ($a \xrightarrow{\mathrm{ttrate}} ∅$):

$\mathrm{ttrate} = \gamma(a) * (c_r + c_t + c_m + c_q)$

All other rates are according to mass action kinetics.

So as an example this is how the model might start out:

```julia
growth_model = @reaction_network begin

    # ribosome binding - this is a very simple example where the rate is defined using mass action kinetics with only one rate constant
    r * mr * kb             mm + r --> rmm
    r * mt * kb             mt + r --> rmt
    r * mm * kb             mm + r --> rmm
    r * mq * kb             mq + r --> rmq

    # dilution - this is a more complicated example, because lam is actually a more complex term
    r * lam                                                              r --> ∅       # so instead of writing this
    r * ((rmr + rmt + rmm + rmq)*(gmax) * a / ((Kgamma ) + a))/mass      r --> ∅       # we would actually need to write this
    ...

end
```

You can see those rates and reactions were just taken straight out of the table above. But as highlighted, some of the rates are complex. It would be useful here to define functions for the rates that we could use in the model. Let's use the growth rate ($\lambda$) as an example:

From above, we can see that the growth rate is made up of a few terms, we can write this as a function to be reused in our code. We can also see from the equations above that $\gamma(a)$ will also need to be reused, so we will start off by defining this as a function.

```julia
function gamma(a, gmax, Kgamma) # the items in brackets are function arguments, we pass them into the function, when we pass them in, the order must be the same as defined.
    return gmax * a / ((Kgamma ) + a)
end
```
We can also write this simple function in another form:
```julia
gamma(a, gmax, Kgamma) = gmax * a / (Kgamma + a)
```

Now we can use this function in our growth rate function:
```julia
lam(a, gmax, Kgamma, cr, ct, cm, cq, mass) = (gamma(a, gmax, Kgamma) * (cr + ct + cm + cq)) / mass
```

We may also want to create functions for other rates, especially when they are similar.

```julia
ribo_bind(mx, r, kb) = r * mx * kb # we can replace mx with the mRNA in the reaction (eg. mq)
```

So now, our model from above will look like this: 

```julia
growth_model = @reaction_network begin

    # ribosome binding
    ribo_bind(mr, r, kb)                                mm + r --> rmm
    ribo_bind(mt, r, kb)                                mt + r --> rmt
    ribo_bind(mm, r, kb)                                mm + r --> rmm
    ribo_bind(mq, r, kb)                                mq + r --> rmq

    # dilution 
    r * lam(a, gmax, Kgamma, cr, ct, cm, cq, mass)      r --> ∅
    ...

end
```

The code you see in these blocks is not being run, so you will need to copy it to a code cell, or write a new version yourself.

Try to build the entire model now, defining functions for rates and using them in the model.


In [ ]:
# build the model here

You could also have a go at writing the ODEs out (like in the Michaelis-Menten example) and see if these match those given by the Catalyst model. To be sure you have written the model correctly in Catalyst you can continue with this notebook reproducing figures.

The parameters for the model are given in the SI, build a dictionary of parameters to use in your model.

In [ ]:
params = Dict()

The initial conditions for the model are given in the below code block:

In [ ]:
u0 = Dict(:mt => 0.0, :mm => 0.0, :mq => 0.0, :mr => 0.0,
          :ct => 0.0, :cm => 0.0, :cq => 0.0, :cr => 0.0,
          :et => 0.0, :em => 0.0, :q => 0.0, :r => 10.0,
          :si => 0.0, :a => 1000.0)

Create an ODEProblem and solve it (you will need to use a stiff solver for this model, ```Rodas5()``` works). In this work, we are often interested in what happens at steady state (when the system is stable over time), therefore, you should use a long time span.

In [ ]:
# solve the model here

<details>
<summary>Click to check your model solution</summary>

![alt text](https://raw.githubusercontent.com/hhindley/CompSysBio_Weisse/3ed46c0ddbd1fdae749b3b821abdcc9b90f7d77e/imgs/model_solution.png "Reactions")

We will now see what can be done with such a model, such as reproducing some bacterial growth laws. We will begin with Monod's law. Vary the nutrient (s0) and solve the model for each value, calculate the growth rate and plot the results:

In [ ]:
nutrient = range(0, 1e4, 100) # values of s0 to simulate
growth_rate = [] # to store growth rates
params_new = deepcopy(params) # make a copy of params to modify s0 so we don't change the original params, remember to use params_for_sim to convert back to list

# for loop to simulate for each nutrient value

# plot the results 

<details>
<summary>Click to check your solution to Monod's law</summary>

![alt text](https://raw.githubusercontent.com/hhindley/CompSysBio_Weisse/3ed46c0ddbd1fdae749b3b821abdcc9b90f7d77e/imgs/monod.png "Reactions")

Now let's look at the nutrient quality growth law: the growth rate is linearly proportional to the ribosomal mass fraction (ΦR) upon increasing nutrient quality. We will need a function to calculate the ribosomal mass fraction:

In [ ]:
calc_rmf(r, cr, ct, cm, cq, nr, Mref) = nr * (r + cr + ct + cm + cq) / Mref

For different values of nutrient quality (ns) solve the model and calculate the associated growth rate and ribosomal mass fraction:

In [ ]:
ns_arr2 = [0.08,0.11541599,0.16651064,0.24022489,0.3466,0.5] # ns values

# for loop to simulate for each ns value

# plot the results 

<details>
<summary>Click to check your model solution</summary>

![alt text](https://raw.githubusercontent.com/hhindley/CompSysBio_Weisse/3ed46c0ddbd1fdae749b3b821abdcc9b90f7d77e/imgs/ns_law.png "Reactions")

We can extend this model to account for the action of ribosome targeting antibiotics. We need to add some new reactions to the model to account for antibiotic binding and dilution of these new species:

$$ c_x \xrightarrow{\mathrm{abx} \cdot k_{cm}} zm_x, $$

$$ zm_x \xrightarrow{\lambda} ∅. $$

In total, we add 4 zombie complexes, 1 for each protein sector, that consist of the drug bound to the ribosome-mRNA complex and 2 new parameters, the antibiotic binding rate ($k_{cm}$) and the concentration of antibiotic ($\mathrm{abx}$).

In [ ]:
# build the abx model here 


In [ ]:
# new parameters for abx model
params_abx = deepcopy(params)
params_abx[:abx] = 0
params_abx[:kcm] = 0.005990373118888

# new initial conditions
u0_abx = deepcopy(u0)
u0_abx[:zmm] = 0.0
u0_abx[:zmr] = 0.0
u0_abx[:zmt] = 0.0
u0_abx[:zmq] = 0.0

Now we can reproduce the translation inhibtion growth law: the addition of a translation-inhibtiing drug results in a negatively proportional relationship between the growth rate and the ribosomal mass fraction. 

Define a new function for the ribosomal mass fraction which includes the new zombie complexes:

In [ ]:
calc_rmf_abx(r, cr, ct, cm, cq, zmr, zmt, zmm, zmq, nr, Mref) = nr * (r + cr + ct + cm + cq + zmr + zmt + zmm + zmq) / Mref

For different values of nutrient quality (ns) and chloramphenicol concentration (Cm) solve the model and calculate the associated growth rate and ribosomal mass fraction:

In [ ]:
ns_arr2 = [0.08,0.11541599,0.16651064,0.24022489,0.3466,0.5]
Cm_arr = [0, 2, 4, 8, 12]
params_abx_new = deepcopy(params_abx)

# for loop to simulate model for each ns and Cm value

# plot the results